In [1]:
# run the single model file fitted on 9:4:6 split data

# %run -i "C:/Users/LL/Desktop/RPI/24_Spring/AMLF/HW_5_SHAP_LIME/Enet_NeuralNet_v2.ipynb"

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('C:/users/LL/Documents/GitHub/AMLF_projects/data.csv')

df = df.drop(columns = ['Unnamed: 0'])

# According to note 30: "Therefore, to predict returns at month t+1, we use most recent monthly characteristics at the end of month t." <br>
# Hence, **shift return t+1 to serve as response: r(t+1)**.

df['r(t+1)'] = df.groupby('permno')['return'].shift(-1)

### handle missing data

# According to note 30 (bottom of p 2248): "Another issue is missing characteristics, which we replace with the cross-sectional median at each month for each stock, respectively." <br>
# Hence, calculate monthly cross-sectional median for features: **'mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr'**.

df_filled = df.copy()
for feature in ['mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr']:
    df_filled[feature] = df_filled.groupby('Date')[feature].transform(lambda x: x.fillna(x.median()))

df_filled.isna().sum()

df.loc[:, ['mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr']] = df_filled.loc[:,['mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr']]

df['Date'] = pd.to_datetime(df['Date'])

# Set the datetime column as index
df.set_index('Date', inplace=True, drop = True)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df_scaled = scaler.fit_transform(df)

df_scaled = pd.DataFrame(df_scaled, columns=df.columns)


permno = df['permno'].reset_index(drop = True)

df_scaled['permno'] = permno

df_scaled.index = df.index

df_scaled_2 = df_scaled.drop(columns = [ 'permno', 'return'])


### split data

##split training, validation, and testing datasets

#training : validation : testing = 6 yr : 4yr : 9 yr <br>
#Also drop the first and last month due to the absence of r(t+1) and return(t-1)

training = df_scaled_2[:'2007-01-01'].dropna()
validation = df_scaled_2['2007-01-01':'2011-01-01']
testing = df_scaled_2['2011-01-01':].dropna()

training_combined = df_scaled_2[:'2011-01-01'].dropna()


##separate X and y

X_train = training.drop(columns = ['r(t+1)'])
y_train = training['r(t+1)']

X_val = validation.drop(columns = ['r(t+1)'])
y_val = validation['r(t+1)']

X_test = testing.drop(columns = ['r(t+1)'])
y_test = testing['r(t+1)']

X_train_combined = training_combined.drop(columns = ['r(t+1)'])
y_train_combined = training_combined['r(t+1)']

In [3]:
## NN

import keras
from keras import layers
from keras import Sequential

# convert dataframes to numpy array

X_train_nn = np.asarray(X_train.values)
y_train_nn = np.asarray(y_train.values)

X_val_nn = np.asarray(X_val.values)
y_val_nn = np.asarray(y_val.values)

X_test_nn = np.asarray(X_test.values)
y_test_nn = np.asarray(y_test.values)

X_train_combined_nn = np.asarray(X_train_combined.values)
y_train_combined_nn = np.asarray(y_train_combined.values)

## model
{'epoch': 100, 'learning_rate': 0.001, 'batch_size': 2500} 

In [4]:
import tensorflow as tf
# import tensorflow_addons as tfa
import random

def r2_score_wo_demeaning_nn(y_true, y_pred):
    ss_res = tf.reduce_sum(tf.square(y_true - y_pred))
    ss_tot = tf.reduce_sum(tf.square(y_true - 0))
    r2 = 1 - (ss_res / ss_tot)
    return r2

In [5]:
y_test_nn.mean()

-0.021094502635160946

In [6]:
from sklearn.metrics import r2_score

In [7]:
random_seed = 77

np.random.seed(random_seed)
tf.random.set_seed(random_seed)
random.seed(random_seed)

# nn3 model

model_nn3 = Sequential([
            layers.Dense(32, activation='relu', input_dim=20),
            layers.Dense(16, activation='relu'),
            layers.Dense(8, activation='relu'),
            layers.Dense(1, activation='linear')
        ])

model_nn3.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                      loss='mean_squared_error', metrics=r2_score_wo_demeaning_nn)

history = model_nn3.fit(X_train_combined_nn, y_train_combined_nn, epochs=100, batch_size=2500,
                            validation_data=(X_test_nn, y_test_nn), verbose=0)


In [8]:
# history.history

In [9]:
last_epoch_metric = history.history['val_r2_score_wo_demeaning_nn'][-1]

last_epoch_metric

-0.0419025793671608

# HW 5

In [10]:
model = model_nn3

In [11]:
X = df_scaled_2.dropna().drop(columns = ['r(t+1)'])

sort data:<br>
by dolvol<br>

In [12]:
X = X.sort_values(by=["dolvol"]).reset_index(drop = True)

In [13]:
print(min, X['dolvol'].min())
print(max, X['dolvol'].max())

<built-in function min> -4.749755120876968
<built-in function max> 1.9821470349015913


create 5 bins

In [14]:
bins = list()
end = 0
start = 0
size = int(113000/5)

for i in range(5):
    end = end + size
    tmp = X.iloc[start:end,:]
    bins.append(tmp)
    start = start + size

In [15]:
# to make sure bins are correct

for item in bins:
    print(len(item))
    print(item['dolvol'].max())
    print(item['dolvol'].min())
    print('------')

22600
-0.9093693679629887
-4.749755120876968
------
22600
-0.17561548513264147
-0.9093631545240612
------
22600
0.35791880105283025
-0.1756097149360904
------
22600
0.8891905133723533
0.3579209861792339
------
22600
1.9821470349015913
0.8892152394947396
------


## LIME
features of interest: dolvol, baspred, sp

In [16]:
import lime
import lime.lime_tabular

In [17]:
X.columns.tolist()

['mom1m',
 'mom12m',
 'chmom',
 'indmom',
 'mom36m',
 'turn',
 'mvel1',
 'dolvol',
 'ill',
 'zerotrade',
 'baspread',
 'retvol',
 'idiovol',
 'beta',
 'betasq',
 'ep',
 'sp',
 'agr',
 'nincr',
 'return(t-1)']

In [18]:
# indexs
idx_1 = [random.randint(0, 22599) for _ in range(10)]

In [19]:
X_np = np.array(X)

**explainer**

In [20]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_np, feature_names=X.columns.tolist(), verbose=True, mode='regression')

**explains**

In [21]:
# idx = idx_1[0]
np.array(bins[0].iloc[idx,])

NameError: name 'idx' is not defined

In [ ]:
explains_1 = list()
for i in range(10):
    idx = idx_1[i]
    exp = explainer.explain_instance(np.array(bins[0].iloc[idx,]), model.predict, num_features=5)
    explains_1.append(exp)

In [ ]:
explains_2 = list()
for i in range(10):
    idx = idx_1[i]          
    exp = explainer.explain_instance(np.array(bins[1].iloc[idx,]), model.predict, num_features=5)
    explains_2.append(exp)

In [ ]:
explains_3 = list()
for i in range(10):
    idx = idx_1[i]
    exp = explainer.explain_instance(np.array(bins[2].iloc[idx,]), model.predict, num_features=5)
    explains_3.append(exp)

In [ ]:
explains_4 = list()
for i in range(10):
    idx = idx_1[i]
    exp = explainer.explain_instance(np.array(bins[3].iloc[idx,]), model.predict, num_features=5)
    explains_4.append(exp)

In [ ]:
explains_5 = list()
for i in range(10):
    idx = idx_1[i]
    exp = explainer.explain_instance(np.array(bins[4].iloc[idx,]), model.predict, num_features=5)
    explains_5.append(exp)

**interpret results**

**bin 1**<br>
For bin 1, mom12m > .34, mom36m > .27, -0.03 < mom1m <= 0.43, retvol > 0.29, nincr <= -0.76, ill > -0.11 have negative impact on return predictions.

In [ ]:
for i in range(10):
    explains_1[i].show_in_notebook(show_table=True)

**bin 2**<br>
For bin 2, turn > 0.25, -0.70 < idiovol <= -0.27, retvol > 0.29, mom1m > -0.03, turn > 0.25, mom36m > 0.27, chmom <= -0.45, ep <= 0.09, mom12m > 0.34 have negative impact on return predictions.

In [ ]:
for i in range(10):
    explains_2[i].show_in_notebook(show_table=True)

**bin 3**<br>
For bin 3, idiovol <= -0.27, mom36m > 0.27, return(t-1) <= -0.44, mom12m > 0.34, chmom <= -0.01, nincr <= -0.76, betasq > 0.27, -0.03 < mom1m, turn > 0.25, return(t-1) <= -0.44, -0.45 < sp <= -0.31 have negative impact on return predictions.

In [ ]:
for i in range(10):
    explains_3[i].show_in_notebook(show_table=True)

**bin 4**<br>
For bin 4, mom36m > 0.27, idiovol <= -0.27, return(t-1) <= -0.44, mom12m > 0.34, mvel1 > -0.17, -0.58 < indmom <= 0.46, turn > 0.25, retvol > 0.29, dolvol > 0.74, -0.01 < chmom <= 0.42, -0.03 < mom1m <= 0.43, nincr <= -0.76 have negative impact on return predictions.

In [ ]:
for i in range(10):
    explains_4[i].show_in_notebook(show_table=True)

**bin 5**<br>
For bin 5, idiovol <= -0.27, mvel1 > -0.17, return(t-1) <= -0.44, dolvol > 0.74, mom1m > -0.03, retvol > 0.29, ep <= 0.09, turn > 0.25, mom12m > 0.34, mom36m > 0.27, chmom <= -0.45,  have negative impact on return predictions.

In [ ]:
for i in range(10):
    explains_5[i].show_in_notebook(show_table=True)

In [ ]:
# exp.as_list()

## Shapley
features of interest: dolvol(8), baspred(11), sp(17)

In [ ]:
from itertools import combinations
import math

In [ ]:

M = 10  # the number of random samples to calculate marginal contribution from
n_features = len(X.columns)
feature_idxs = [7, 10, 16]
sample_to_explain = list()
shapley_value_all = list()


for df in bins:  # for each bin
    samplex = df.sample(10)
    sample_to_explain.append(samplex)
    xarray = samplex.values
    
    for i in range(10):
        x = xarray[i]
        print("selected instance: ", x)# instance of interest
        shapley_values = []

        for i in range(3):   # for each feature of interest
            x_idx = [7, 10, 16]
            marg_contri_feature = []

            feature_of_interest = feature_idxs[i]
            print("Index of feature of interest: ", feature_of_interest)
            x_idx.remove(feature_of_interest)

            all_combinations = []  # all coalitions of features minus the feature of interest
            for r in range(1, len(x_idx) + 1):
                combinations_of_length_r = list(combinations(x_idx, r))
                all_combinations.extend(combinations_of_length_r)

            for coalition in all_combinations:  # for each coalition
                coalition = list(coalition)
                print('the current coalition: ', coalition)
                weight = math.factorial(len(coalition)) * math.factorial(n_features - len(coalition)-1)/ math.factorial(n_features)
                print("weight of this coalition: ", weight)

                marg_contri_coalition = []  # marginal contributions of this coalition

                for _ in range(M):
                    z = df.sample(1).values[0]  # random select sample    

                    # construct two new instances
                    x_orig = np.array([x[i] if i in coalition + [feature_of_interest] else z[i] for i in range(n_features)])
                    x_rando = np.array([x[i] if i in coalition else z[i] for i in range(n_features)])             

                    # calculate marginal contribution
                    marginal_contribution = model.predict(x_orig.reshape(1, -1))[0][0] - model.predict(x_rando.reshape(1, -1))[0][0]
                    marg_contri_coalition.append(marginal_contribution)

                sigma_j = sum(marg_contri_coalition) / len(marg_contri_coalition)
                shapley_coalition = sigma_j*weight  # for one coaltion examined for feature of interest
                marg_contri_feature.append(shapley_coalition)

            shapley_j = sum(marg_contri_feature)  # shapley value of feature j
            shapley_values.append(shapley_j)

            print("feature of interest: ", feature_of_interest)
            print("shapley value of this feature: ", shapley_j)
            print('--------')
            print('--------')
        shapley_value_all.append(shapley_values)

In [ ]:
all_sample = pd.concat([sample_to_explain[0], sample_to_explain[1]], ignore_index=True)
all_sample = pd.concat([all_sample, sample_to_explain[2]], ignore_index=True)
all_sample = pd.concat([all_sample, sample_to_explain[3]], ignore_index=True)
all_sample = pd.concat([all_sample, sample_to_explain[4]], ignore_index=True)

In [ ]:
all_sample

In [ ]:
shapleydf = pd.DataFrame(shapley_value_all, columns = ['feature: dolvol', 'feature: baspred', 'feature: sp'])

In [ ]:
shapley_mtx = pd.concat([all_sample, shapleydf], axis=1)

### shapley value matrix with all samples examined
bin1: 0-9<br>
bin2: 10-19<br>
bin3: 20-29<br>
bin4: 30-39<br>
bin5: 40-49<br>

In [ ]:
shapley_mtx

**bin 1**<br>
dolvol generally has a positive impact on return prediction, baspred and sp have negative ones.

In [ ]:
bin1_shap = shapley_mtx.iloc[0:10,]

In [ ]:
bin1_shap.loc[:,['feature: dolvol', 'feature: baspred', 'feature: sp']].describe()

**bin 2**<br>
sp generally has a positive impact on return prediction, baspred and dolvol have negative ones.

In [ ]:
bin2_shap = shapley_mtx.iloc[10:20,]

In [ ]:
bin2_shap.loc[:,['feature: dolvol', 'feature: baspred', 'feature: sp']].describe()

**bin 3**<br>
dolvol generally has a positive impact on return prediction, baspred and sp have negative ones.

In [ ]:
bin3_shap = shapley_mtx.iloc[20:30,]

In [ ]:
bin3_shap.loc[:,['feature: dolvol', 'feature: baspred', 'feature: sp']].describe()

**bin 4**<br>
baspred and dolvol generally has a positive impact on return prediction, sp have negative ones.

In [ ]:
bin4_shap = shapley_mtx.iloc[30:40,]

In [ ]:
bin4_shap.loc[:,['feature: dolvol', 'feature: baspred', 'feature: sp']].describe()

**bin 5**<br>
sp and dolvol generally has a positive impact on return prediction, baspred have negative ones.

In [ ]:
bin5_shap = shapley_mtx.iloc[40:50,]

In [ ]:
bin5_shap.loc[:,['feature: dolvol', 'feature: baspred', 'feature: sp']].describe()

**overall**<br>
Overall, dolvol have a positive impact on return predictions, baspred and sp have negative impact but the standard deviation are larger, indicating different local impacts.

In [ ]:
shapley_mtx.loc[:,['feature: dolvol', 'feature: baspred', 'feature: sp']].describe()